In [2]:
# Installing Interactive Brokers API

# This is not installed with pip, instead you must install TWS or IB Gateway 
# then install the TWS API and install via the following commands
# `python -m pip install setuptools`
# `cd C:\TWS API\source\pythonclient\`
# `python setup.py install`
import ibapi

In [6]:
# Collect S&P 500 historic data
import logging
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import pandas as pd
import threading

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.historical_data_end = threading.Event()  # Completion flag

    def historicalData(self, reqId, bar):
        logging.info(f"Received data for {bar.date}")
        self.data.append([bar.date, bar.close, bar.open, bar.volume, bar.high, bar.low])

    def historicalDataEnd(self, reqId, start, end):
        logging.info(f"Historical data request ended for reqId: {reqId}")
        self.historical_data_end.set()  # Set the flag when data ends

def run_loop():
    app.run()

# Establish connection to IB
app = IBApi()
app.connect("127.0.0.1", 7497, clientId=123)
logging.info("Connecting to Interactive Brokers API...")
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

# Wait for the connection to be established
while not app.isConnected():
    logging.info("Waiting for connection...")
    threading.Event().wait(1)  # Wait for 1 second before checking again

# Define the contract
contract = Contract()
contract.symbol = "SPY"
contract.secType = "STK"
contract.exchange = "SMART"
contract.currency = "USD"
logging.info("Contract defined for SPY.")

# Request historical data
# docs: https://interactivebrokers.github.io/tws-api/historical_bars.html
app.reqHistoricalData(
    reqId=1, 
    contract=contract,
    endDateTime='', 
    durationStr='30 Y',
    barSizeSetting='1 day', 
    whatToShow='TRADES',
    useRTH=True, 
    formatDate=1, 
    keepUpToDate=False,
    chartOptions=[]
    )
logging.info("Requested historical data for SPY.")

# Wait for historical data to be fetched
if not app.historical_data_end.wait(timeout=60):  # Timeout after 60 seconds
    logging.warning("Timed out waiting for historical data.")
else:
    logging.info("Historical data request completed.")

# Disconnect
app.disconnect()
logging.info("Disconnected from Interactive Brokers API.")

# Create DataFrame
price_df = pd.DataFrame(app.data, columns=['Date', 'Close', 'Open', 'Volume', 'High', 'Low'])
print(price_df)


2023-12-28 13:41:40,976 - INFO - sent startApi
2023-12-28 13:41:40,978 - INFO - REQUEST startApi {}
2023-12-28 13:41:40,979 - INFO - SENDING startApi b'\x00\x00\x00\n71\x002\x00123\x00\x00'
2023-12-28 13:41:40,980 - INFO - ANSWER connectAck {}
2023-12-28 13:41:40,981 - INFO - Connecting to Interactive Brokers API...
2023-12-28 13:41:40,984 - INFO - ANSWER managedAccounts {'accountsList': 'DU7625017'}
2023-12-28 13:41:40,987 - INFO - Contract defined for SPY.
2023-12-28 13:41:40,988 - INFO - ANSWER nextValidId {'orderId': 1}
2023-12-28 13:41:40,990 - INFO - ANSWER error {'reqId': -1, 'errorCode': 2104, 'errorString': 'Market data farm connection is OK:usfarm.nj', 'advancedOrderRejectJson': ''}
2023-12-28 13:41:40,989 - INFO - REQUEST reqHistoricalData {'reqId': 1, 'contract': 1910618814944: 0,SPY,STK,,0,,,SMART,,USD,,,False,,,,combo:, 'endDateTime': '', 'durationStr': '30 Y', 'barSizeSetting': '1 day', 'whatToShow': 'TRADES', 'useRTH': True, 'formatDate': 1, 'keepUpToDate': False, 'char

          Date   Close    Open    Volume    High     Low
0     19940104   46.66   46.53    164300   46.66   46.47
1     19940105   46.75   46.72    710900   46.78   46.53
2     19940106   46.75   46.81    201000   46.84   46.69
3     19940107   47.03   46.84    775500   47.06   46.72
4     19940110   47.59   47.09    593700   47.59   46.97
...        ...     ...     ...       ...     ...     ...
7543  20231221  472.70  471.32  57760976  472.98  468.84
7544  20231222  473.65  473.86  44344028  475.38  471.70
7545  20231226  475.65  474.08  31409570  476.58  473.99
7546  20231227  476.51  475.44  39490581  476.66  474.89
7547  20231228  477.13  476.84  15508514  477.50  476.43

[7548 rows x 6 columns]


In [9]:
import logging
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import pandas as pd
import threading
import datetime

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = {}
        self.historical_data_end = threading.Event()

    def historicalData(self, reqId, bar):
        logging.info(f"Received data for {bar.date}")
        if reqId not in self.data:
            self.data[reqId] = []
        self.data[reqId].append([bar.date, bar.close])

    def historicalDataEnd(self, reqId, start, end):
        logging.info(f"Historical data request ended for reqId: {reqId}")
        self.historical_data_end.set()

    def requestFuturesData(self, contract, reqId, duration):
        self.reqHistoricalData(
            reqId=reqId, 
            contract=contract,
            endDateTime='', 
            durationStr=duration,
            barSizeSetting='1 day', 
            whatToShow='TRADES',
            useRTH=True, 
            formatDate=1, 
            keepUpToDate=False,
            chartOptions=[]
        )

    def process_historical_data(self):
        data_1mo = pd.DataFrame(self.data[1], columns=['Date', '1moFuture'])
        data_6mo = pd.DataFrame(self.data[2], columns=['Date', '6moFuture'])

        data_1mo['Date'] = pd.to_datetime(data_1mo['Date'])
        data_6mo['Date'] = pd.to_datetime(data_6mo['Date'])

        combined_data = pd.merge(data_1mo, data_6mo, on='Date', how='outer')
        return combined_data

def defineFuturesContract(symbol, expiry):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = "FUT"
    contract.exchange = "GLOBEX"
    contract.currency = "USD"
    contract.lastTradeDateOrContractMonth = expiry
    return contract

def run_loop():
    app.run()

# Establish connection to IB
app = IBApi()
app.connect("127.0.0.1", 7497, clientId=123)
logging.info("Connecting to Interactive Brokers API...")
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

# Wait for the connection to be established
while not app.isConnected():
    logging.info("Waiting for connection...")
    threading.Event().wait(1)

# Request historical data for each futures contract for the past 20 years
current_date = datetime.datetime.now()
start_date = current_date - datetime.timedelta(days=365 * 20)
dates_range = pd.date_range(start=start_date, end=current_date, freq='M')

for date in reversed(dates_range):
    one_month_expiry = date.strftime("%Y%m")
    six_months_expiry = (date + pd.DateOffset(months=6)).strftime("%Y%m")

    one_month_contract = defineFuturesContract("ES", one_month_expiry)
    six_months_contract = defineFuturesContract("ES", six_months_expiry)

    app.requestFuturesData(one_month_contract, reqId=1, duration='1 M')
    app.requestFuturesData(six_months_contract, reqId=2, duration='6 M')

    threading.Event().wait(1.5)

if not app.historical_data_end.wait(timeout=120):
    logging.warning("Timed out waiting for historical data.")
else:
    logging.info("Historical data request completed.")
    combined_df = app.process_historical_data()
    print(combined_df)

app.disconnect()
logging.info("Disconnected from Interactive Brokers API.")


2023-12-28 14:24:24,023 - INFO - sent startApi
2023-12-28 14:24:24,024 - INFO - REQUEST startApi {}
2023-12-28 14:24:24,025 - INFO - SENDING startApi b'\x00\x00\x00\n71\x002\x00123\x00\x00'
2023-12-28 14:24:24,027 - INFO - ANSWER connectAck {}
2023-12-28 14:24:24,028 - INFO - Connecting to Interactive Brokers API...
2023-12-28 14:24:24,037 - INFO - REQUEST reqHistoricalData {'reqId': 1, 'contract': 1910619020256: 0,ES,FUT,202311,0,,,GLOBEX,,USD,,,False,,,,combo:, 'endDateTime': '', 'durationStr': '1 M', 'barSizeSetting': '1 day', 'whatToShow': 'TRADES', 'useRTH': True, 'formatDate': 1, 'keepUpToDate': False, 'chartOptions': []}
2023-12-28 14:24:24,037 - INFO - ANSWER managedAccounts {'accountsList': 'DU7625017'}
2023-12-28 14:24:24,038 - INFO - SENDING reqHistoricalData b'\x00\x00\x00D20\x001\x000\x00ES\x00FUT\x00202311\x000.0\x00\x00\x00GLOBEX\x00\x00USD\x00\x00\x000\x00\x001 day\x001 M\x001\x00TRADES\x001\x000\x00\x00'
2023-12-28 14:24:24,043 - INFO - ANSWER nextValidId {'orderId': 1

KeyboardInterrupt: 

2023-12-28 14:56:25,350 - INFO - ANSWER error {'reqId': -1, 'errorCode': 2104, 'errorString': 'Market data farm connection is OK:usopt', 'advancedOrderRejectJson': ''}
2023-12-28 14:56:25,351 - ERROR - ERROR -1 2104 Market data farm connection is OK:usopt
2023-12-28 14:59:06,341 - INFO - ANSWER error {'reqId': -1, 'errorCode': 2108, 'errorString': 'Market data farm connection is inactive but should be available upon demand.usopt', 'advancedOrderRejectJson': ''}
2023-12-28 14:59:06,342 - ERROR - ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
2023-12-28 14:59:06,343 - INFO - ANSWER error {'reqId': -1, 'errorCode': 2108, 'errorString': 'Market data farm connection is inactive but should be available upon demand.usopt', 'advancedOrderRejectJson': ''}
2023-12-28 14:59:06,344 - ERROR - ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usopt
